In [13]:
from tqdm import tqdm
tqdm.pandas()

import pandas as pd
import numpy as np

In [2]:
import os, json
import pickle
import base64
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


In [2]:
import cohere
import uuid
from typing import List, Dict

In [3]:
def get_credentials():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds


In [4]:
get_credentials()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=157082210581-s4tar0624afsvf9cr3g61mm13lt01l80.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53258%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=yDGo9WCyn4lBNzjYL8SeSxENhhB0ll&access_type=offline


In [5]:
def get_emails(service, labels):
    emails = []
    for label in labels:
        results = service.users().messages().list(userId='me', labelIds=[label], maxResults=500).execute()
        if 'messages' in results:
            emails.extend(results['messages'])
        while 'nextPageToken' in results:
            page_token = results['nextPageToken']
            results = service.users().messages().list(userId='me', labelIds=[label], pageToken=page_token, maxResults=500).execute()
            if 'messages' in results:
                emails.extend(results['messages'])
    return emails


In [7]:
creds = get_credentials()
service = build('gmail', 'v1', credentials=creds)

labels = ['INBOX']  # Add more labels if needed
emails = get_emails(service, labels)


In [10]:
links = set()
for email in emails:
    msg = service.users().messages().get(userId='me', id=email['id']).execute()
    payload = msg['payload']
    headers = payload['headers']
    for header in headers:
        if header['name'] == 'Subject':
            subject = header['value']
        if header['name'] == 'From':
            sender = header['value']
        if header['name'] == 'Date':
            date = header['value']
    
    print(subject)
    # print(payload)
    try:        
        if 'parts' in payload:
            for part in payload['parts']:
                if part['mimeType'] == 'text/plain':
                    message_body = base64.urlsafe_b64decode(part['body']['data']).decode('utf-8')
                    break
        else:
            message_body = base64.urlsafe_b64decode(payload['body']['data']).decode('utf-8')
    except:
        message_body = ""
        print(f"SKIPPING: {subject}")

    break


[2410.02536] Intelligence at the Edge of Chaos


In [19]:
import re
def extract_first_url(text):
    url_pattern = r'https?://\S+'
    match = re.search(url_pattern, text)
    return match.group(0) if match else None

In [12]:
message_body.strip()\
    .replace('Thanks,\r\nShubham', '')\
    .replace("Thanks and regards,\r\nShubham Chandel", "")\
    .replace("Thanks and Regards,\r\n\r\nShubham Chandel\r\nNew York University", "")\
    .replace("-- \r\nThis message (including any attachments) contains confidential information \r\nintended for a specific individual and purpose, and is protected by law. If \r\nyou are not the intended recipient, you should delete this message and are \r\nhereby notified that any disclosure, copying, or distribution of this \r\nmessage, or the taking of any action based on it, is strictly prohibited.", "")\
    .replace("<", "").replace(">", "").replace("\r\n", "")


'https://arxiv.org/abs/2410.02536https://www.arxiv.org/abs/2410.02536'

In [11]:
co = cohere.Client(api_key="e7FTxIVWjp0NEHTvIg34DYK89kE62P5mITf0FNqw")

In [19]:
# from unstructured.partition.html import partition_html
# from unstructured.chunking.title import chunk_by_title

# def load_and_chunk(row):
#     """
#     Loads the text from the sources and chunks the HTML content.
#     """
#     try:
#         elements = partition_html(url=row["link"])
#         chunks = chunk_by_title(elements, max_characters=2000)
#         return [str(chunk) for chunk in chunks]
#     except:
#         print(row["link"])
#         return []

In [37]:
df = pd.read_json("./links.jsonl", lines=True)

In [38]:
df.shape

(894, 3)

In [36]:
# df['prompt'] = df.apply(lambda x: f"Title: {x.subject}\nLink: {x.link}\n" + "\n".join(x.docs[:20]), axis=1)
df['prompt'] = df.apply(lambda x: f"Title: {x.subject}\nLink: {x.link}", axis=1)
df['embeddings'] = co.embed(texts=list(df.prompt.values), model="embed-english-v3.0", input_type="search_document").embeddings


In [24]:
df.to_pickle("df.embedding")

In [29]:
df = pd.read_pickle("df.embedding")
df.shape

(858, 5)

In [33]:
import requests

term = "jax learn"
url = "https://sksq96--search-app-searchapp-search-dev.modal.run"
params = {"term": term}

response = requests.get(url, params=params)
response.raise_for_status()  # Raise an exception for bad status codes
len(response.json())

858